In [1]:
#!pip install transformers
#ImportError: You need to install sacremoses to use BioGptTokenizer. See https://pypi.org/project/sacremoses/ for installation.
#!pip install sacremoses
#!pip install torch
#!pip install pandas
#!pip install tensorflow

In [1]:
import torch
from transformers import BioGptForCausalLM, BioGptTokenizer, Trainer, TrainingArguments
import pandas as pd
from torch.utils.data import Dataset 
import datetime
from torch.utils.data import random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import time

/mmfs1/home/6672islamk/venv37/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-08 20:36:42.561715: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 20:36:44.258357: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64:/cm/local/apps/gcc/9.2.0/lib:/cm/local/apps/gcc/9.2.0/lib64


In [2]:
data = pd.read_csv(r'./Data/train_small.csv')
data = pd.read_csv(r'./Data/train.csv')

In [3]:
data['Chief Complaint'] = data['Chief Complaint'].apply(lambda x: x.replace(u'\xa0',u' '))
data['Chief Complaint'] = data['Chief Complaint'].apply(lambda x: x.replace('\u200a',' '))

all_sentences = data['Chief Complaint'].values.tolist()

In [4]:
print("sample size : ",len(all_sentences))
print("samples     : " )
all_sentences[0:10]

sample size :  7593
samples     : 


['Patient reportedly cut her LLE while putting on a compression sock, patient was intially hypotensive and c/o generalized weakness, bleeding controlled pta.',
 'fall from standing at home. left jaw abrasion, left shoulder pain.',
 '"The blood pressure medicine they gave me makes me throw up", pmhx:  HTN. bipolar',
 'hearing voices, depression, pt. states "the devil is trying to get me". hyperreligious/talking to self during triage assessment.',
 'Throat ache, hemoptysis, blood in urine, PMH NAFLD',
 'pain in groin, scrotum and lower back, swelling in scrotum x 2 days. burning w/ urination. pmh: sleep apnea',
 'R sided HA starting "a few minutes ago". Denies blurry vision or dizziness, also denies trauma. Hx: HIV',
 'c/o  bright red bloody bowel movement at 0405. c/o r sided abd cramping assoc with it. denies pmh',
 'swallowed a sip of hydrogen peroxide about 2041. family reports he was crying and "turning red". now alert & calm',
 'flu symptoms chills, low grade fevers, nausea/vomitin

In [5]:
model = BioGptForCausalLM.from_pretrained("microsoft/biogpt")
tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")

In [7]:
#tokenizer some samples
print( tokenizer.encode("ED") )
print( tokenizer.encode("AM") )
print( tokenizer.encode("swelling") )
print( tokenizer.encode("twisting ankle") )
print( tokenizer.encode("upper/right") )

[2, 2448]
[2, 3332]
[2, 4609]
[2, 6854, 32693, 4725]
[2, 1349, 26, 702]


In [6]:
max_len = max([len(tokenizer.encode(s)) for s in all_sentences])

print(f"max_len {max_len}")

max_len 86


In [7]:
#mark beginning and end of sentences with with sos and eos
def tokenize_seq(sent,tokenizer,max_length):
  return tokenizer('<sos>'+ sent + '<eos>', truncation=True, max_length=max_length, padding="max_length")

class CCDataset(Dataset):

  def __init__(self, sentences, tokenizer, gpt_type="bio-gpt", max_length=max_len):

    self.tokenizer = tokenizer 
    self.input_ids = []
    self.attn_masks = []

    for sentence in sentences:      
      encodings = tokenize_seq(sentence,tokenizer,max_length)
            
      self.input_ids.append(torch.tensor(encodings['input_ids']))
      self.attn_masks.append(torch.tensor(encodings['attention_mask']))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]   

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))   

In [8]:
import gc
gc.collect() 

42

In [9]:
#create an instance of Dataset
dataset = CCDataset(all_sentences, tokenizer, max_length=max_len)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_set, val_set = random_split(dataset, [train_size, val_size])
print("train_size :",train_size)
print("val_size   :",val_size)

gc.collect() 

train_size : 6833
val_size   : 760


0

In [30]:
dataset[0]

(tensor([    2,   109,  4514,   279,   361,  2974, 24902,  3427,  3109,  7219,
           316,   247, 32580,    25,    14,  3341,   578, 12704,     7,   125,
            17,   501, 24857, 13709,     8,   457,    26,   399,  3891,  7052,
             7,  1720,   691,   341,  4011,     4,   109,   768,  2842,   361,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [12]:
train_dataloader = DataLoader(train_set,  sampler = RandomSampler(train_set), batch_size = 32)
validation_dataloader = DataLoader(val_set, sampler = SequentialSampler(val_set), batch_size = 32 )

In [11]:
import tensorflow as tf
import torch

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    print('GPU device not found')

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('Use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

GPU device not found
No GPU available, using the CPU instead.


2023-05-08 20:37:43.966081: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 20:37:44.094458: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-08 20:37:44.094507: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (raj-ln1): /proc/driver/nvidia/version does not exist


In [13]:
optimizer = torch.optim.Adam(model.parameters(),lr = 0.0005)
model = model.to(device)

In [14]:
#at every step i want to check if generations are getting better.
def eval_keywords(model, keywords):
  model.eval()
  for keyword in keywords:
    input_seq = "<sos> " + keyword
    generated = torch.tensor(tokenizer.encode(input_seq)).unsqueeze(0)
    generated = generated.to(device)
    sample_outputs = model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=30, 
                                max_length = 100,
                                top_p=0.90, 
                                num_return_sequences=2
                                )
    for i, sample_output in enumerate(sample_outputs):
      print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

#keywords = ["swollen tonsils, LMP 10/19/98 have","complaints of blood in","swollen tonsils, LMP 10/19/98 have not","complaints of blood in stool within the"]

In [22]:
#at every step i want to check if generations are getting better.
def eval_keywords(keywords):
  model.eval()
  for keyword in keywords:
    input_seq = "<sos> " + keyword
    generated = torch.tensor(tokenizer.encode(input_seq)).unsqueeze(0)
    generated = generated.to(device)
    sample_outputs = model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=30, 
                                max_length = 100,
                                top_p=0.90, 
                                num_return_sequences=2
                                )
    for i, sample_output in enumerate(sample_outputs):
      print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

keywords = ["swollen tonsils, LMP 10/19/98 have","complaints of blood in","swollen tonsils, LMP 10/19/98 have not","complaints of blood in stool within the"]

In [14]:
#call model with a batch of input
def process_one_batch(batch):
  b_input_ids = batch[0].to(device)
  b_labels = batch[0].to(device)
  b_masks = batch[1].to(device)
  outputs  = model(b_input_ids,  attention_mask = b_masks,labels=b_labels)
  return outputs, b_input_ids

#do one epoch for training
def train_epoch():
  t0 = time.time()
  total_train_loss = 0
  model.train()
  for step, batch in enumerate(train_dataloader):
        
        model.zero_grad()        
        outputs, input_ids = process_one_batch( batch)
        loss = outputs[0]  
        batch_loss = loss.item()
        total_train_loss += batch_loss

        loss.backward()
        optimizer.step()

        
  avg_train_loss = total_train_loss / len(train_dataloader)  
  print("avg_train_loss",avg_train_loss)  
  elapsed_time = format_time(time.time() - t0)
  print("elapsed time for 1 training epoch : ",elapsed_time)

#do one epoch for eval
def eval_epoch():
  t0 = time.time()
  total_eval_loss = 0
  nb_eval_steps = 0
  # Evaluate data for one epoch
  for batch in validation_dataloader:            
        
    with torch.no_grad():        
      outputs, input_ids = process_one_batch( batch)
      loss = outputs[0]              
      batch_loss = loss.item()
      total_eval_loss += batch_loss
      logits = outputs.logits

  # Compute the perplexity
  log_probs = torch.nn.functional.log_softmax(logits[0, :-1], dim=-1)
  target = input_ids[0, 1:]
  perplexity = torch.exp(-log_probs.gather(dim=-1, index=target.unsqueeze(-1)).sum() / target.size(0))
  print("Perplexity:", perplexity.item())

  avg_val_loss = total_eval_loss / len(validation_dataloader)
  print("avg_val_loss",avg_val_loss) 
  elapsed_time = format_time(time.time() - t0)
  print("elapsed time for 1 eval epoch : ",elapsed_time)

In [16]:
model_path = './Trained_models/bioGPT_small/'
model_path = './Trained_models/bioGPT_large/'

# For small dataset

avg_train_loss 4.880961608886719

elapsed time for 1 training epoch :  0:04:02

Perplexity: 13.712059020996094

avg_val_loss 2.2061687111854553

elapsed time for 1 eval epoch :  0:00:14

avg_train_loss 1.8096208175023396

elapsed time for 1 training epoch :  0:03:25

Perplexity: 9.81473445892334

avg_val_loss 1.8228430151939392

elapsed time for 1 eval epoch :  0:00:12

# For Large dataset

avg_train_loss 1.585690320652222

elapsed time for 1 training epoch :  0:49:10

Perplexity: 5.314290523529053

avg_val_loss 1.2847954332828522

elapsed time for 1 eval epoch :  0:03:17

avg_train_loss 1.049023493706623

elapsed time for 1 training epoch :  0:48:53

Perplexity: 5.240231990814209

avg_val_loss 1.2551174809535344

elapsed time for 1 eval epoch :  0:03:04



In [37]:
train_epoch()
eval_epoch()
train_epoch()
eval_epoch()
model.save_pretrained(model_path)

avg_train_loss 1.585690320652222
elapsed time for 1 training epoch :  0:49:10
Perplexity: 5.314290523529053
avg_val_loss 1.2847954332828522
elapsed time for 1 eval epoch :  0:03:17
avg_train_loss 1.049023493706623
elapsed time for 1 training epoch :  0:48:53
Perplexity: 5.240231990814209
avg_val_loss 1.2551174809535344
elapsed time for 1 eval epoch :  0:03:04


In [18]:
keywords = ["swollen tonsils, LMP 10/19/98 have","complaints of blood in","swollen tonsils, LMP 10/19/98 have not","complaints of blood in stool within the"]

In [19]:
model = BioGptForCausalLM.from_pretrained(model_path)
eval_keywords( model, keywords )

0: < sos > swollen tonsils, LMP 10 / 19 / 98 have been seen at MMC. given levaquin at home and given levaquin for flu. not improving. < eos >
1: < sos > swollen tonsils, LMP 10 / 19 / 98 have been seen x 2 days for same; hx adhd, seizures < eos >
0: < sos > complaints of blood in urine, urination x 1 week; PMH: Hep-C, Hep-C < eos >
1: < sos > complaints of blood in urine, lower back pain, chest pain, low back pain x 4 days; PMH: HTN < eos >
0: < sos > swollen tonsils, LMP 10 / 19 / 98 have not been feeling better. PMH: HTN < eos >
1: < sos > swollen tonsils, LMP 10 / 19 / 98 have not felt "something" per mom "< eos >
0: < sos > complaints of blood in stool within the stool; "I've had my pancreas removed this AM. I just had my pancreas removed & it did so, I had my pancreas removed." denies any complaints; < eos >
1: < sos > complaints of blood in stool within the stool last night, N / V and D x1 day. PMH: COPD, HTN, DM, HLD < eos >


In [23]:
train_epoch()
eval_epoch()
eval_keywords( keywords )

avg_train_loss 2.2862285653750103
elapsed time for 1 training epoch :  0:05:15
avg_val_loss 2.288579225540161
elapsed time for 1 eval epoch :  0:00:14
0: < sos > swollen tonsils, LMP 10 / 19 / 98 have x/ HTN, / eos >
1: < sos > swollen tonsils, LMP 10 / 19 / 98 have for PMPMx /.
0: < sos > complaints of blood in > HTN / PMH, Hx / PMmmxx mx. PMPMH: eos >
1: < sos > complaints of blood in PMH.
0: < sos > swollen tonsils, LMP 10 / 19 / 98 have not ies.
1: < sos > swollen tonsils, LMP 10 / 19 / 98 have not eos >
0: < sos > complaints of blood in stool within the x. PMH to > eos >
1: < sos > complaints of blood in stool within the eos >


In [32]:
train_epoch()
eval_epoch()
eval_keywords( keywords )

avg_train_loss 0.5711676220099131
elapsed time for 1 training epoch :  0:03:30
avg_val_loss 2.0633411407470703
elapsed time for 1 eval epoch :  0:00:14
0: < sos > swollen tonsils, LMP 10 / 19 / 98 have a mass in chest on same side pushing against spine < eos >. sent from PCP abnormal asymptomatic PMHx: negative for < eos >
1: < sos > swollen tonsils, LMP 10 / 19 / 98 have gone through 4 weeks and 5 days < eos > erwith hn, left heart pain < eos >. States was placed on for being. hx ter ter in triage. last chemo in < eos >
0: < sos > complaints of blood in urine since last night, pt states she was in withdrawals now. pt states she feels like she's in withdrawals now. PMHX: depression, anxiety < eos >, HTN < eos >
1: < sos > complaints of blood in face, chest tightness x 1 week < eos > esting and not feeling like they heard "in ess. Hx endocarditis in Son in Ancef in Car 26, 07. < eos >
0: < sos > swollen tonsils, LMP 10 / 19 / 98 have not been having surgery due to LMP 1 year ago < eos >

In [33]:
train_epoch()
eval_epoch()
eval_keywords( keywords )

avg_train_loss 0.4855570097764333
elapsed time for 1 training epoch :  0:03:30
avg_val_loss 2.053557336330414
elapsed time for 1 eval epoch :  0:00:14
0: < sos > swollen tonsils, LMP 10 / 19 / 98 have gone through 4 pads- G1 LMG 10 / 27 and 2013. Denies other symptoms < eos >. pmh < eos >. < eos >
1: < sos > swollen tonsils, LMP 10 / 19 / 98 have had abdominal surgery and couldn't void at home. PMH: cervical fusion, right ear drainage. pmh: asthma < eos >, right ear pain < eos >
0: < sos > complaints of blood in urine since last night, states that her last night has had blood in urine since then. Hit is out of blood pressure medication; denies cough. Also c / o back pain in lower back, denies dysuria. < eos > SIS w / d, c / o intermittent pain in
1: < sos > complaints of blood in lower back and fluid on L buttocks. Pt. pt states that she feels like her's going to the clinic and she states that she feels like she has gone through blood clots in lower back. pta. Pt denies blood clots. Pt

In [3]:
#keywords = ["swollen tonsils, LMP 10/19/98 have","complaints of blood in","swollen tonsils, LMP 10/19/98 have not","complaints of blood in stool within the"]

In [15]:
keywords = ["Pelvic pain and vaginal bleeding X 1 wk. Soaking a pad about every 2 hours. Syncopal episode yesterday, denies", "left knee pain with movement for past 2 weeks"]

In [16]:
model = BioGptForCausalLM.from_pretrained(model_path)
eval_keywords( model, keywords )

0: < sos > Pelvic pain and vaginal bleeding X 1 wk. Soaking a pad about every 2 hours. Syncopal episode yesterday, denies hitting head, denies PMH, + ETOH < eos >
1: < sos > Pelvic pain and vaginal bleeding X 1 wk. Soaking a pad about every 2 hours. Syncopal episode yesterday, denies nausea / vomitting, + LOC < eos >
0: < sos > left knee pain with movement for past 2 weeks. PMH: DM, Hep C < eos >
1: < sos > left knee pain with movement for past 2 weeks, also c / o knee pain x1 month. pmhx: chronic back pain, diabetes, HTN < eos >


In [22]:
keywords = ["Pelvic pain and vaginal bleeding", "left knee pain with movement for past 2 weeks", "severe abdominal pain for 3 days. recently admitted for h pylori, states"]

In [41]:
# Left thumb injury with needle nose pliers. Bleeding controlled
# Returned for rabies
# left knee pain with movement for past 2 weeks post workout routine, PMH- High Cholesterol
# swelling and redness to left side of face, pt states that there was a
# C/O back pain after falling backwards onto back from 6ft porch, denies LOC or any head or nack pain. States it is
# States it is hard to catch his breath PMHX bipolar schizophrenia
#  Has tried Imtrex and nasal
# Headahce, fever, chills, bodyaches, non productive cough since this AM. PMH HTN
keywords = ["severe abdominal pain for"]

In [42]:
model = BioGptForCausalLM.from_pretrained(model_path)
eval_keywords( model, keywords )

0: < sos > severe abdominal pain for the past few days, denies pmh < eos >
1: < sos > severe abdominal pain for 2 weeks with nausea, vomiting and diarrhea x 3 days; hx chole < eos >
